# 9 - Tick Priips with inputs (C#)

### Setup and import packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: CsvHelper"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files 
using CsvHelper;
using System.IO;
using System.Threading;
using System.Globalization;

### Retrieve the token

In [ ]:
var headers = TokenHelpers.GetTokenAsMetaData();

### Load inputs

In [ ]:
var inputFilename = "input_equities_filtered.csv";

In [ ]:
// Input trade structure to load from the csv file
public class InputTrade
{
    public string GUID { get; set; } // Guid format doesn't work
    public string ISIN { get; set; } 
    public string INSTR_DESC { get; set; } 
    public string TRA_TYPE { get; set; }
    public double EXECUTION_PRICE { get; set; }
    public string ORDER_TIMESTAMP { get; set; }
    public string EXECUTION_TIMESTAMP { get; set; }
    public string OUR_TICKER { get; set; }
    public string OUR_EXCHANGE { get; set; }
}

In [ ]:
// Load the trades from the csv input file
var records = new List<InputTrade>();
using (var reader = new StreamReader(inputFilename))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    records = csv.GetRecords<InputTrade>().ToList();
}

### Process Request
Read the input file, process the request for each row, and write results in the output file

In [ ]:
#!time
var outputFilename = "equities_output.csv";
var field = PriipsPrice.Trade;
var latency = new Duration { Seconds = 1 * 60 };

using (var fs = new StreamWriter(File.Create(outputFilename))) 
{
    fs.WriteLine($"Guid,Isin,Instrument_descriptor,Type,ArrivalTime,ArrivalPrice,ArrivalCost,OpenTime,CloseTime,OrderTime,ExecutionTime,Duration (s),Latency (s),ExecutionPrice,Count,OpenPrice,HighPrice,LowPrice,ClosePrice,Vwap,Volume,Ticker,Exchange");
    var channel = ChannelHelpers.GetChannel();
    foreach(var record in records)
    {
        
        var ot = DateTime.Parse(record.ORDER_TIMESTAMP).ToUniversalTime();
        var et = DateTime.Parse(record.EXECUTION_TIMESTAMP).ToUniversalTime();
        if (ot.Day != et.Day || ot.TimeOfDay > et.TimeOfDay)
        {
            continue;
        }
        var request = new TickPriipsRequest 
                                        { 
                                            Identifier = new Identifier { Exchange = record.OUR_EXCHANGE , Ticker = record.OUR_TICKER},
                                            OrderTime = Timestamp.FromDateTime(ot),
                                            ExecutionTime = Timestamp.FromDateTime(et),
                                            ExecutionPrice = record.EXECUTION_PRICE,
                                            Field = field,
                                            Latency = latency
                                        };

        var service = new TickPriipsService.TickPriipsServiceClient(channel);
        var call = service.TickPriips(request, headers);
        await foreach (var current in call.ResponseStream.ReadAllAsync())
        {
            var guid = record.GUID;
            var ticker = record.OUR_TICKER;
            var exchange = record.OUR_EXCHANGE;
            var isin = record.ISIN;
            var instr_descriptor = record.INSTR_DESC;
            var trade_type = record.TRA_TYPE;
            var arrivalTime = current.ArrivalTime;
            var arrivalPrice = current.ArrivalPrice;
            var arrivalCost = current.ArrivalCost;
            var openTime = current.OpenTime;
            var closeTime = current.CloseTime;
            var orderTime = current.OrderTime;
            var exectutionTime = current.ExecutionTime;
            var duration = current.Duration.Seconds;
            var latency = current.Latency.Seconds;
            var executionPrice = current.ExecutionPrice;
            var count = current.Count;
            var openPrice = current.OpenPrice;
            var highPrice = current.HighPrice;
            var lowPrice = current.LowPrice;
            var closePrice = current.ClosePrice;
            var vwap = current.Vwap;
            var volume = current.Volume;
            
            if (openPrice > 0) 
            {
                fs.WriteLine($"{guid},{isin},{instr_descriptor},{trade_type},{arrivalTime},{arrivalPrice},{arrivalCost},{openTime},{closeTime},{orderTime},{exectutionTime},{duration},{latency},{executionPrice},{count},{openPrice},{highPrice},{lowPrice},{closePrice},{vwap},{volume},{ticker},{exchange}");

            }
        }
    }
}
Console.WriteLine($"Done exporting data ({records.Count} records)");